In [ ]:
##Data preprocessing step I- Calculate the presence of each core gene (nucleotide sequence) in the number of strains 

import pandas as pd
import os
import stat
import statistics
import numpy as np
import timeit
from Bio import SeqIO
import subprocess
import sys

MAIN_DIR='/Users/arcsan/Documents/allele_from_pan_genome_data_analysis/data/data1/pan_gene/Lactobacillus/Lactobacillus_helveticus/pangenome_alignments/'
path='/Users/arcsan/Documents/allele_from_pan_genome_data_analysis/data/data1/pan_gene/Lactobacillus/Lactobacillus_helveticus/alleleome/'

df=pd.read_csv(path + 'df_pangene_summary_v2.csv')
core_gene_list = (df['pangenome_class_2'].eq('Core').groupby(df['Gene']).any()).pipe(lambda x:x.index[x].tolist())
genes_df = pd.DataFrame()
for k in core_gene_list:
    s=0
    i=0
    nuc_allele_file = MAIN_DIR + k + '/input/'
    nuc_file=  nuc_allele_file + 'pangenes.fna'
    f=open(nuc_file,'r+')
    for seq_record in SeqIO.parse(nuc_file, "fasta"):
        g_id=seq_record.id
        l_gene=len(seq_record)
        i=i+1
        s=s+l_gene
    df=pd.DataFrame.from_dict({
    "Gene":k,
    "Number_of_strains":i,
    }, orient='index')
    df=df.T
    genes_df=pd.concat([df,genes_df])
    genes_df.to_csv(os.path.join(path,'Genes_nuc_length_number_of_strains.csv'))




In [ ]:
#Data preprocessing step II- Extract only the genes which are present in greater than 5% of strains giving the set of genes to be further studied
import numpy as np
import pandas as pd
import os

MAIN_DIR='/Users/arcsan/Documents/allele_from_pan_genome_data_analysis/data/data1/pan_gene/Lactobacillus/Lactobacillus_helveticus/pangenome_alignments/'
path='/Users/arcsan/Documents/allele_from_pan_genome_data_analysis/data/data1/pan_gene/Lactobacillus/Lactobacillus_helveticus/alleleome/'

df=pd.read_csv(path +'Genes_nuc_length_number_of_strains.csv')
df_genes=df[df.Number_of_strains < 2].index
df.drop(df_genes, inplace=True)
df.to_csv(os.path.join(path,'Final_nuc_genes_present_in_above_5_percent_of_strains.csv'))


In [ ]:
#Data preprocessing stepIII- Get the locus tag and length of each allele (nucleotide sequence) of the set of genes
import pandas as pd
import os
import stat
import statistics
import numpy as np
from Bio import SeqIO
import subprocess
import sys

MAIN_DIR='/Users/arcsan/Documents/allele_from_pan_genome_data_analysis/data/data1/pan_gene/Lactobacillus/Lactobacillus_helveticus/pangenome_alignments/'
path='/Users/arcsan/Documents/allele_from_pan_genome_data_analysis/data/data1/pan_gene/Lactobacillus/Lactobacillus_helveticus/alleleome/'

df=pd.read_csv(path +'df_pangene_summary_v2.csv')
core_gene_list = (df['pangenome_class_2'].eq('Core').groupby(df['Gene']).any()).pipe(lambda x:x.index[x].tolist())
genes_df = pd.DataFrame()
for k in core_gene_list:
    s=0
    gene_length=[]
    i=0
    nuc_allele_file = MAIN_DIR + k + '/input/'
    nuc_file=  nuc_allele_file + 'pangenes.fna'
    f=open(nuc_file,'r+')
    for seq_record in SeqIO.parse(nuc_file, "fasta"):
        g_id=seq_record.id
        l_gene=len(seq_record)
        gene_length.append(l_gene)
        i=i+1
        s=s+l_gene
        df=pd.DataFrame.from_dict({
        "Gene":k,
        "Locus_tag":g_id,    
        "Length_of_allele":l_gene,    
        }, orient='index')
        df=df.T
        genes_df=pd.concat([df,genes_df])
        genes_df.to_csv(os.path.join(path,'Genes_nuc_length_of_alleles.csv'))




In [ ]:
#Data preprocessing stepIV- Calculate mean length, standard deviation of the length of the set of genes
import pandas as pd
import os

MAIN_DIR='/Users/arcsan/Documents/allele_from_pan_genome_data_analysis/data/data1/pan_gene/Lactobacillus/Lactobacillus_helveticus/pangenome_alignments/'
path='/Users/arcsan/Documents/allele_from_pan_genome_data_analysis/data/data1/pan_gene/Lactobacillus/Lactobacillus_helveticus/alleleome/'

df=pd.read_csv(path + 'Genes_nuc_length_of_alleles.csv')
new_df = df.groupby(['Gene']).Length_of_allele.agg({'mean','std'})
new_df['mean']=new_df['mean'].apply(lambda x:round(x,0))
new_df['std']=new_df['std'].apply(lambda x:round(x,2))
new_df.to_csv(os.path.join(path,'core_nuc_alleles_with_mean_std.csv'))
df = df.merge(new_df, left_on=['Gene'], right_index=True)
df.to_csv(os.path.join(path,'core_nuc_alleles_with_locus_mean_std.csv'))

In [ ]:
#Data preprocessing stepV-Get the genes with their alleles with length less than two standard deviation of the mean length 
import pandas as pd
import os

MAIN_DIR='/Users/arcsan/Documents/allele_from_pan_genome_data_analysis/data/data1/pan_gene/Lactobacillus/Lactobacillus_helveticus/pangenome_alignments/'
path='/Users/arcsan/Documents/allele_from_pan_genome_data_analysis/data/data1/pan_gene/Lactobacillus/Lactobacillus_helveticus/alleleome/'
df1=pd.read_csv(path +'core_nuc_alleles_with_locus_mean_std.csv')
df1['mean_2std']=(df1['mean']- 2*df1['std'])
df2=df1[df1.Length_of_allele < df1.mean_2std]
df2.to_csv(os.path.join(path,'core_alleles_with_length_less_than_2std_less_than_mean_length.csv'))


In [ ]:
#Data preprocessing stepVI-Removal of amino acid sequences with specific locus tag with length which is less than two standard deviation of the mean length from the pangenome and create a new file of final set of alleles (of amino acid sequences).In this step
#the new file is created only of those genes in which alleles with prescribed length conditions are matched. 
import glob
from webbrowser import get
from Bio.Blast import NCBIXML
import pandas as pd
from Bio.Seq import Seq
import os
from Bio import SeqIO
import sys

MAIN_DIR='/Users/arcsan/Documents/allele_from_pan_genome_data_analysis/data/data1/pan_gene/Lactobacillus/Lactobacillus_helveticus/pangenome_alignments/'
path='/Users/arcsan/Documents/allele_from_pan_genome_data_analysis/data/data1/pan_gene/Lactobacillus/Lactobacillus_helveticus/alleleome/'

df=pd.read_csv(path + 'core_alleles_with_length_less_than_2std_less_than_mean_length.csv')
gene_locus_list=df['Gene'].to_list()
gene_locus_list=set(gene_locus_list)
locus_list=df['Locus_tag'].to_list()
sequences=[]
for s in gene_locus_list:
    aa_allele_path = MAIN_DIR + s + '/input/' 
    aa_file=  aa_allele_path + 'pangenes.faa'
    new_file= ''.join(aa_allele_path + 'pan_genes.faa')
    f=open(aa_file,'r')
    f1=open(new_file,'w')
    for seq_record in SeqIO.parse(aa_file, "fasta"):
        if seq_record.id not in locus_list:
            print(seq_record.id)
            desc=seq_record.description
            allele_seq=str(seq_record.seq)
            alleles=''.join(['>'+desc + '\n'+ allele_seq+'\n'])
            f1.write(alleles)
f1.close()

In [ ]:
#Data preprocessing stepVII- Rename the unlatered gene file names for further workflow execution.In the previous step, only those genes in which the allelels with less length doesn't exist , their new files will not be created.Hence it is necessary to change the file names of unlatered genes.
import os
import pandas as pd

MAIN_DIR='/Users/arcsan/Documents/allele_from_pan_genome_data_analysis/data/data1/pan_gene/Lactobacillus/Lactobacillus_helveticus/pangenome_alignments/'
path='/Users/arcsan/Documents/allele_from_pan_genome_data_analysis/data/data1/pan_gene/Lactobacillus/Lactobacillus_helveticus/alleleome/'
df=pd.read_csv(path + 'df_pangene_summary_v2.csv')
core_gene_list = (df['pangenome_class_2'].eq('Core').groupby(df['Gene']).any()).pipe(lambda x:x.index[x].tolist())
#dataframe of core gene alleles with genes not satisfying the condition to be removed
df1=pd.read_csv(path + 'core_alleles_with_length_less_than_2std_less_than_mean_length.csv')
edit_list=df1['Gene'].to_list()
new_gene_list=set(edit_list)
for f in core_gene_list:
    print(f)
    if f not in new_gene_list:
        print(f)
        aa_allele_path = MAIN_DIR + f + '/input/' 
        aa_file=  aa_allele_path + 'pangenes.faa'
        new_file= aa_allele_path + 'pan_genes.faa'
        os.rename(*(os.path.join(aa_allele_path, fname) for fname in (aa_file, new_file)))

In [ ]:
#Data preprocessing stepVIII-Create new output directory to save the results of the further processing.
import pandas as pd
import os
import random
import pathlib
from os.path import join, getsize

MAIN_DIR='/Users/arcsan/Documents/allele_from_pan_genome_data_analysis/data/data1/pan_gene/Lactobacillus/'
species_list=['Lactobacillus_paragasseri', 'Lacticaseibacillus_rhamnosus', 'Weissella_cibaria', 'Lactobacillus_crispatus', 'Oenococcus_oeni', 'Pediococcus_pentosaceus', 'Pediococcus_acidilactici', 'Lactobacillus_acidophilus', 'Ligilactobacillus_ruminis', 'Lacticaseibacillus_paracasei', 'Lactobacillus_gasseri', 'Levilactobacillus_brevis', 'Lactobacillus_delbrueckii', 'Latilactobacillus_sakei', 'Lentilactobacillus_parabuchneri', 'Limosilactobacillus_reuteri', 'Leuconostoc_inhae', 'Leuconostoc_mesenteroides', 'Lactobacillus_johnsonii', 'Lactiplantibacillus_pentosus', 'Ligilactobacillus_salivarius','Lactiplantibacillus_plantarum','Lactobacillus_helveticus', 'Lactobacillus_iners', 'Weissella_confusa', 'Limosilactobacillus_fermentum']
j=0
for k in species_list:
    MAIN_DIR_1= MAIN_DIR + k + '/pangenome_alignments/'
    query_list_1=[name for name in os.listdir(MAIN_DIR_1) if os.path.isdir(os.path.join(MAIN_DIR_1, name))]
    for i in query_list_1:
        root_dir = os.path.join(MAIN_DIR_1 + i +'/')
        dir="output"
        path = os.path.join(root_dir, dir)
        j=j+1
        os.makedirs(path)

In [ ]:
#Alleleome generation step I- Get the consensus of the allels (amino acid sequance) of each gene in the core pangenome

import pandas as pd
import os
from Bio import AlignIO
from pathlib import Path
from Bio.Align import AlignInfo
from Bio import SeqIO
from Bio.Blast import NCBIWWW
import stat
from Bio.Blast import NCBIXML
from Bio import SearchIO
import numpy as np
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align.Applications import MafftCommandline
import subprocess
from io import StringIO
import collections
from collections import Counter
import sys

MAIN_DIR='/Users/arcsan/Documents/allele_from_pan_genome_data_analysis/data/data1/pan_gene/Lactobacillus/'
species_list=['Lactobacillus_paragasseri', 'Lacticaseibacillus_rhamnosus', 'Weissella_cibaria', 'Lactobacillus_crispatus', 'Oenococcus_oeni', 'Pediococcus_pentosaceus', 'Pediococcus_acidilactici', 'Lactobacillus_acidophilus', 'Ligilactobacillus_ruminis', 'Lacticaseibacillus_paracasei', 'Lactobacillus_gasseri', 'Levilactobacillus_brevis', 'Lactobacillus_delbrueckii', 'Latilactobacillus_sakei', 'Lentilactobacillus_parabuchneri', 'Limosilactobacillus_reuteri', 'Leuconostoc_inhae', 'Leuconostoc_mesenteroides', 'Lactobacillus_johnsonii', 'Lactiplantibacillus_pentosus', 'Ligilactobacillus_salivarius','Lactiplantibacillus_plantarum','Lactobacillus_helveticus', 'Lactobacillus_iners', 'Weissella_confusa', 'Limosilactobacillus_fermentum']
for i in species_list:
    MAIN_DIR_1= MAIN_DIR + i + '/alleleome/'
    MAIN_DIR_2= MAIN_DIR + i + '/pangenome_alignments/'
    df=pd.read_csv(MAIN_DIR_1 + 'df_pangene_summary_v2.csv')
    core_aa_query_list = (df['pangenome_class_2'].eq('Core').groupby(df['Gene']).any()).pipe(lambda x:x.index[x].tolist())
    for k in core_aa_query_list:
        aa_allele_file = MAIN_DIR_2 + k + '/input/' 
        aa_file=  aa_allele_file + 'pan_genes.faa'
        if os.stat(aa_file).st_size == 0:
            print("The file is empty")
            pass
        else:
            mafft_cline = MafftCommandline(input=aa_file)
            stdout, stderr = mafft_cline()
            mafft_out_file=''.join(MAIN_DIR_2+ k +'/output/'+ 'mafft_thresh_aa_' + k +'.fasta' )
            with open(mafft_out_file, "w") as handle:
                handle.write(stdout)
            myalign = AlignIO.read(mafft_out_file, "fasta")
            summary = AlignInfo.SummaryInfo(myalign)
            consensus = summary.dumb_consensus(threshold=0.5)
            seq=str(consensus)
            seq=seq.upper()
            consensus_aa_file = os.path.join(MAIN_DIR_2 + k + '/input/'+ 'consensus_nuc_thresh'+ k +'.faa')
            consensus_aa = open(consensus_aa_file, 'w+')
            consensus_seq= ''.join(['>' + k +'\n'+ seq])        
            consensus_aa.write(consensus_seq)
            consensus_aa.close()


In [ ]:
#Alleleome generation step II- Alignment of each allele sequnces (amino acid sequence) of each gene with consensus sequence using BLASTp.
import pandas as pd
import os
import random
import pathlib
from os.path import join, getsize

MAIN_DIR='/Users/arcsan/Documents/allele_from_pan_genome_data_analysis/data/data1/pan_gene/Lactobacillus/'
species_list=['Lactobacillus_paragasseri', 'Lacticaseibacillus_rhamnosus', 'Weissella_cibaria', 'Lactobacillus_crispatus', 'Oenococcus_oeni', 'Pediococcus_pentosaceus', 'Pediococcus_acidilactici', 'Lactobacillus_acidophilus', 'Ligilactobacillus_ruminis', 'Lacticaseibacillus_paracasei', 'Lactobacillus_gasseri', 'Levilactobacillus_brevis', 'Lactobacillus_delbrueckii', 'Latilactobacillus_sakei', 'Lentilactobacillus_parabuchneri', 'Limosilactobacillus_reuteri', 'Leuconostoc_inhae', 'Leuconostoc_mesenteroides', 'Lactobacillus_johnsonii', 'Lactiplantibacillus_pentosus', 'Ligilactobacillus_salivarius','Lactiplantibacillus_plantarum','Lactobacillus_helveticus', 'Lactobacillus_iners', 'Weissella_confusa', 'Limosilactobacillus_fermentum']

for i in species_list:
    MAIN_DIR_1= MAIN_DIR + i + '/alleleome/'
    MAIN_DIR_2= MAIN_DIR + i + '/pangenome_alignments/'
    df=pd.read_csv(MAIN_DIR_1 + 'df_pangene_summary_v2.csv')
    core_aa_query_list = (df['pangenome_class_2'].eq('Core').groupby(df['Gene']).any()).pipe(lambda x:x.index[x].tolist())
    import subprocess
    blast_path='/home/azureuser/datadrive/ncbi-blast-2.13.0+/bin/blastp'
    for r in range(len(core_aa_query_list)):
        query=core_aa_query_list[r]
        if (pd.isnull(query)==False):
            out_file_name = MAIN_DIR_2 + query + '/output/' + 'amino_acid_' + 'blast_out_thresh' + query + '.xml'
            args = (blast_path,
                    '-query', MAIN_DIR_2 + query + '/input/'+'pan_genes.faa',
                    '-subject', MAIN_DIR_2 + query + '/input/'+ 'consensus_aa_thresh'+ query +'.faa' ,
                    '-outfmt', '5',
                )
            with open(out_file_name, 'w+') as outfile:
                subprocess.run(args, stdout=outfile)

In [ ]:
#Alleleome generation step III- Parsing the results and generating the amino acid mutations
import pandas as pd
import numpy as np
import os
from Bio import pairwise2
from Bio.Seq import Seq
from Bio.pairwise2 import format_alignment

MAIN_DIR='/Users/arcsan/Documents/allele_from_pan_genome_data_analysis/data/data1/pan_gene/Lactobacillus/'
species_list=['Lactobacillus_paragasseri', 'Lacticaseibacillus_rhamnosus', 'Weissella_cibaria', 'Lactobacillus_crispatus', 'Oenococcus_oeni', 'Pediococcus_pentosaceus', 'Pediococcus_acidilactici', 'Lactobacillus_acidophilus', 'Ligilactobacillus_ruminis', 'Lacticaseibacillus_paracasei', 'Lactobacillus_gasseri', 'Levilactobacillus_brevis', 'Lactobacillus_delbrueckii', 'Latilactobacillus_sakei', 'Lentilactobacillus_parabuchneri', 'Limosilactobacillus_reuteri', 'Leuconostoc_inhae', 'Leuconostoc_mesenteroides', 'Lactobacillus_johnsonii', 'Lactiplantibacillus_pentosus', 'Ligilactobacillus_salivarius','Lactiplantibacillus_plantarum','Lactobacillus_helveticus', 'Lactobacillus_iners', 'Weissella_confusa', 'Limosilactobacillus_fermentum']

# String parsing helper functions
# fast find all chars in string: https://stackoverflow.com/questions/52452911/finding-all-positions-of-a-character-in-a-string
def _find_all_idx(string, character):
    idx = string.find(character)
    while idx != -1:
        yield idx
        idx = string.find(character, idx + 1)
        
def _get_char_idxs_in_str(s, c):
    char_idxs = list(_find_all_idx(s, c))
    char_idxs = [x for x in char_idxs]
    return char_idxs
    

CHAR_OF_INTEREST = '-'
assert(_get_char_idxs_in_str('AR-D-G', CHAR_OF_INTEREST)==[2,4])

# BLAST results glossary: https://www.ncbi.nlm.nih.gov/books/NBK62051/
# Description of special characters on the match string: https://blast.ncbi.nlm.nih.gov/Blast.cgi?CMD=Web&PAGE_TYPE=BlastDocs&DOC_TYPE=BlastHelp
# The databases alignments are displayed as pairs of matches between query and subject sequence.
# A middle line between the query and subject sequence displays the status of a letter.
# For protein alignments (e.g, BLASTP/BLASTX/TBLASTN), identities present the letter,
# conservative substitutions present a "+", and nothing otherwise. This is the default view.
# https://www.ncbi.nlm.nih.gov/books/NBK62051/: "If the aligned residues have similar physico-chemical properties or have a
# positive score in the governing scoring matrix the substitution is said to be conservative."
# '-' in subject string indicate an insertion
# BLAST subject and query results string are always the same length.
# Won't find differences in the beginning of genes since doesn't compare query vs subject start and end match positions.
# This logic was removed since alleles can have enormous differences on either of their sequences.
# Best to keep the returned data seperate for finding
# consecutive mutations according to AA positions.

BLAST_INDEL_CHAR = '-'
def _get_ins_d(subject_str, query_str, subject_match_start_pos):
    ins_d = dict()
    for idx in _get_char_idxs_in_str(subject_str, BLAST_INDEL_CHAR):
        pos = idx+subject_match_start_pos
        ins_d[pos] = query_str[idx]
    return ins_d

# Won't find differences in the beginning of genes since doesn't compare query vs subject start and end match positions.
def _get_del_d(subject_str, query_str, subject_match_start_pos):
    del_d = dict()
    for idx in _get_char_idxs_in_str(query_str, BLAST_INDEL_CHAR):
        pos = idx+subject_match_start_pos
        del_d[pos] = subject_str[idx]
    return del_d
    
sbjct = 'AR-D-G'  # insertion @ 3 and 5
qry = '-RBDHG'  # deletion @ 1
assert(_get_ins_d(sbjct, qry, 1)=={3: 'B', 5: 'H'})
assert(_get_ins_d(sbjct, qry, 2)=={4: 'B', 6: 'H'})  # test effects of subject_match_start_pos

assert(_get_del_d(sbjct, qry, 1)=={1: 'A'})
assert(_get_del_d(sbjct, qry, 2)=={2: 'A'})  # test effects of subject_match_start_pos


ALL_AMINO_ACIDS = {'A','R','N','D','B','C','E','Q','Z','G','H','I','L','K','M','F','P','S','T','W','Y','V'}

def _get_sub_aa_d(subject_str, match_str, query_str, indel_aa_pos_l, subject_match_start_pos):
    seq_chng_d = dict()
    
    for c_i in range(0, len(match_str)):
        if match_str[c_i] not in ALL_AMINO_ACIDS:  # finding BLAST characters indicating mismatch or gap
            pos = subject_match_start_pos+c_i  # subject_match_start_pos+c_i since subject_match_start_pos represents the first charater and the index c_i == 0
            if pos not in indel_aa_pos_l:
                seq_chng_d[pos] = {'s':subject_str[c_i], 'q':query_str[c_i]}
    
    return seq_chng_d



assert(_get_sub_aa_d('ARPGCQ', '+ PG Q', 'MTPGBQ', [], 1)=={1: {'s':'A', 'q':'M'}, 2: {'s':'R', 'q':'T'}, 5: {'s':'C', 'q':'B'}})
assert(_get_sub_aa_d('ARPGCQ', '+ PG Q', 'MTPGBQ', [5], 1)=={1: {'s':'A', 'q':'M'}, 2: {'s':'R', 'q':'T'}})  # testing use of indel_aa_pos_l
assert(_get_sub_aa_d('ARPGCQ', '+ PG Q', 'MTPGBQ', [], 2)=={2: {'s':'A', 'q':'M'}, 3: {'s':'R', 'q':'T'}, 6: {'s':'C', 'q':'B'}})  # test effects of subject_match_start_pos



from itertools import groupby
from operator import itemgetter


def _get_consecutive_pos_l(pos_l):
    cnsc_pos_l = []
    for k, g in groupby(enumerate(pos_l), lambda ix : ix[0] - ix[1]):
        cnsc_pos_l.append(list(map(itemgetter(1), g)))
    return cnsc_pos_l

mut_pos_l = [1, 4,5, 7,8,9, 22]
assert(_get_consecutive_pos_l(mut_pos_l)==[[1], [4, 5], [7, 8, 9], [22]])


import glob
from Bio.Blast import NCBIXML
gene_var_df = pd.DataFrame()

for i in species_list:
    MAIN_DIR_1= MAIN_DIR + i + '/alleleome/'
    MAIN_DIR_2= MAIN_DIR + i + '/pangenome_alignments/'
    df=pd.read_csv(MAIN_DIR_1 + 'df_pangene_summary_v2.csv')
    core_aa_query_list = (df['pangenome_class_2'].eq('Core').groupby(df['Gene']).any()).pipe(lambda x:x.index[x].tolist())
    for blast_output_file in core_aa_query_list:
        blast_file_name= MAIN_DIR_2 + blast_output_file + '/output/'
        blast_output_file_path = blast_file_name + 'amino_acid_blast_out_thresh'+ blast_output_file + '.xml'
        gene = blast_output_file.replace(blast_file_name,'').replace('.xml','')
        for record in NCBIXML.parse(open(blast_output_file_path)):
            if len(record.alignments) > 0:
                #Description of available members: https://biopython.org/docs/1.75/api/Bio.Blast.Record.html
                subject_match_start_pos = record.alignments[0].hsps[0].sbjct_start
                blast_subject_str = record.alignments[0].hsps[0].sbjct
                blast_match_str = record.alignments[0].hsps[0].match
                blast_query_str = record.alignments[0].hsps[0].query
                length_align = record.alignments[0].hsps[0].align_length
                align_score = record.alignments[0].hsps[0].score
                num_align= record.alignments[0].hsps[0].num_alignments
                e_value=record.alignments[0].hsps[0].expect
                num_indentities= record.alignments[0].hsps[0].identities
                ref_id= record.alignments[0].hit_id
                ref_info= record.alignments[0].hit_def # new added#### column name to be added####
                query_info= record.query
                query_info=query_info.split("|", 1)
                query_description=query_info[0]
                query_description=query_description.split(" ", 1)
                query_id=query_description[0]
                query_desc=query_description[1]
                GCF_id=query_info[1]
                # Won't find differences in the beginning of genes since doesn't compare query vs subject start and end match positions.
                ins_d = _get_ins_d(blast_subject_str, blast_query_str, subject_match_start_pos)
                for cnsc_pos_l in _get_consecutive_pos_l(ins_d.keys()):
                    seq_chng_str = ''.join([ins_d[p] for p in cnsc_pos_l])
                    df = pd.DataFrame.from_dict({
                        'Gene': gene,
                        "Mutation_source": "Pangenome variant",
                        "AA_start_pos": min(cnsc_pos_l),
                        "AA_end_pos" : max(cnsc_pos_l),
                        "Mutation_size": len(cnsc_pos_l),
                        "AA_mutation_type": "Insertion",
                        'AA_cons_seq': '',
                        "AA_seq_change": seq_chng_str,
                        "Length_alignment": length_align,
                        "Score" : align_score,
                        "E_value": e_value,
                        "Identity" : num_indentities,
                        "Query_locus_tag" : query_id,
                        "Query_description" : query_desc,
                        "GCF_id" : GCF_id,
                        "Sequence_type": "Variant",
                    }, orient='index')  # Can't use columns due to diff parsing of 'AA range' input
                    df = df.T
                    gene_var_df = pd.concat([df, gene_var_df]) # Accounting for multiples of same allele
                # Won't find differences in the beginning of genes since doesn't compare query vs subject start and end match positions.
                del_d = _get_del_d(blast_subject_str, blast_query_str, subject_match_start_pos)
                for cnsc_pos_l in _get_consecutive_pos_l(del_d.keys()):
                    seq_chng_str = ''.join([del_d[p] for p in cnsc_pos_l])
                    df = pd.DataFrame.from_dict({
                            'Gene': gene,
                            "Mutation_source": "Pangenome variant",
                            "AA_start_pos": min(cnsc_pos_l),
                            "AA_end_pos" : max(cnsc_pos_l),
                            "Mutation_size": len(cnsc_pos_l),
                            "AA_mutation_type": "Deletion",
                            'AA_cons_seq': seq_chng_str,
                            "AA_seq_change": '',
                            "Length_alignment": length_align,
                            "Score" : align_score,
                            "E_value": e_value,
                            "Identity" : num_indentities,
                            "Query_locus_tag" : query_id,
                            "Query_description" : query_desc,
                            "GCF_id" : GCF_id,
                            "Sequence_type": "Variant",
                    }, orient='index')  # Can't use columns due to diff parsing of 'AA range' input
                    df = df.T
                    gene_var_df = pd.concat([df, gene_var_df]) # Accounting for multiples of same allele
                sub_d = _get_sub_aa_d(
                        blast_subject_str,
                        blast_match_str,
                        blast_query_str,
                list(ins_d.keys()) + list(del_d.keys()),
                subject_match_start_pos)
                for cnsc_pos_l in _get_consecutive_pos_l(sub_d.keys()):
                    seq_ref_str = ''.join([sub_d[p]['s'] for p in cnsc_pos_l])
                    seq_chng_str = ''.join([sub_d[p]['q'] for p in cnsc_pos_l])
                    df = pd.DataFrame.from_dict({
                            'Gene': gene,
                            "Mutation_source": "Pangenome variant",
                            "AA_start_pos": min(cnsc_pos_l),
                            "AA_end_pos" : max(cnsc_pos_l),
                            "Mutation_size": len(cnsc_pos_l),
                            "AA_mutation_type": "Substitution",
                            'AA_cons_seq': seq_ref_str,
                            "AA_seq_change": seq_chng_str,
                            "Length_alignment": length_align,
                            "Score" : align_score,
                            "E_value": e_value,
                            "Identity" : num_indentities,
                            "Query_locus_tag" : query_id,
                            "Query_description" : query_desc,
                            "GCF_id" : GCF_id,
                            "Sequence_type": "Variant",
                    }, orient='index')  # Can't use columns due to diff parsing of 'AA range' input
                    df = df.T
                    gene_var_df = pd.concat([df, gene_var_df]) # Accounting for multiples of same allele
        gene_var_df.to_csv(os.path.join(MAIN_DIR_1,'pan_aa_gene_vars_df'  + '.csv'))          